In [21]:
import os
import numpy as np
from PIL import Image

In [22]:
# !unzip Test_2_TIG_txt.zip

In [23]:
path="Test_2"
files=os.listdir(path)

In [24]:
def generate_size(no_features,no_pixels):
  cols=no_features*no_pixels*2-no_pixels
  cols=cols+1
  break_point=((no_features-1)*no_pixels)+1
  rows=break_point+1
  return rows,cols,break_point

def generate_array():
  arr=[]
  for i in range(rows):
    col=[]
    for j in range(cols):
      col.append(-1)
    arr.append(col)
  return arr

def generate_index(filename):
  f=open(filename,"r")
  line=f.readlines()
  l1=line[0].split("\t")
  index=[]
  for ele in l1:
    index.append(int(ele))
  return index

def generate_intensity(filename):
  f1=open("Test_2/FTD_TIG_Temporal_txt/"+filename,"r")
  lines=f1.readlines()
  intensity=[]
  for line in lines:
    line1=line.split("\t")
    for ele in line1:
      intensity.append(float(ele))
  return intensity

def create_grid(arr,feature,direction,pixels,no_features,breakpoint,pixel_values,feature_ordering):
  if(direction=="right"):
    start=pixels*(feature)
    intensity=get_pixel(feature,pixel_values,feature_ordering)
    for i in range(1,breakpoint+1):
      start+=1
      for j in range(pixels):
        if(arr[i][j+start]!=-1):
          arr[i][j+start]=0.5*arr[i][j+start]+0.5*intensity
        else:
          arr[i][j+start]=intensity
    return arr
  if(direction=="down"):
    start=pixels*(feature)
    intensity=get_pixel(feature,pixel_values,feature_ordering)
    for i in range(breakpoint,0,-1):
      start+=1
      for j in range(pixels):
        if(arr[i][j+start]!=-1):
          arr[i][j+start]=0.5*arr[i][j+start]+0.5*intensity
        else:
          arr[i][j+start]=intensity
    return arr

def get_pixel(feature,pixel_values,feature_ordering):
  index1=feature_ordering.index(feature)
  return pixel_values[index1]

def generate_grid(index,intensity,arr,no_features,no_of_pixels,break_point):
  for feature in index:
    arr=create_grid(arr,feature,"right",no_of_pixels,no_features,break_point,intensity,index)
    arr=create_grid(arr,feature,"down",no_of_pixels,no_features,break_point,intensity,index)

  for i in range(1,rows):
    for j in range(1,cols):
      if arr[i][j]==-1:
        arr[i][j]=0
  grid=[]
  for i in range(1,rows):
    col=[]
    for j in range(1,cols):
      col.append(arr[i][j])
    grid.append(col)
  return grid

In [ ]:
os.mkdir("ftd_rep1_v2_txt")
os.mkdir("ftd_rep1_v2_img")

In [27]:
path="Test_2/FTD_TIG_Temporal_txt"
files=os.listdir(path)
indexfilename="Test_2/_index.txt"
no_features= 36
no_of_pixels=2
rows,cols,break_point=generate_size(no_features,no_of_pixels)
# arr=generate_array()
index=generate_index(indexfilename)
for each_file in files:
  arr=generate_array()
  filename=each_file
  # l4.append(filename)
  l6=each_file.split(".")
  imagename=l6[0]+".png"
  intensity=generate_intensity(filename)
  grid=generate_grid(index,intensity,arr,no_features,no_of_pixels,break_point)
  file_path="ftd_rep1_v2_txt/"+filename
  np.savetxt(file_path,grid, fmt="%f", delimiter="\t")
  image_path="ftd_rep1_v2_img/"+imagename
  grid_img=np.array(grid,dtype=np.uint8)
  image = Image.fromarray(grid_img)
  image.save(image_path)
